In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/My Drive/Colab Notebooks/COMP0197/comp0197-cw2

/content/drive/My Drive/Colab Notebooks/COMP0197/comp0197-cw2


In [6]:
%ls

'Assessed-Component-23 COMP0197 2022-23 (1).pdf'   __pycache__/
 data/                                             README.md
 data_augmentation.py                              Train.ipynb
 Data_processing_v1.ipynb                          train_notebook.ipynb
 Losses/                                           Train.py
 models/                                           Train-tib.py
 model_UNet.py                                     Train_unsupervised.ipynb
 preprocessing_1_dataloader.ipynb                  unet/
 preprocessing_1_dataloader.py                     unet-dont-use/
 preprocessing_2_dataloaders.py                    Utils/


In [7]:
! pip install torch
! pip install torchvision
! pip install numpy
! pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
import model_UNet
from data_augmentation import augmentation, colorjiter, invert
import matplotlib.pyplot as plt
from preprocessing_1_dataloader import get_data

In [9]:
#### Hyper-Param ####

# device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #Enable GPU support
print(f'Using device: {device}')

# data parms
supervised_percent = 0.25  # what percent of training is to be labelled
img_resize = 64             # resize all images to this size 
is_mixed_labels = True     # Training only on supervised data or not

# model params
depth = 3       # depth of unet
dropout_rate = 0.25

# Training params
batch_size = 32
epochs = 20
ramp_up = 10
consistency = 56
alpha = 0.999
global_step = 0
lr = 1e-3
lr_gamma = 0.9
wait_period = 5

Using device: cpu


In [10]:
a = torch.arange(12).reshape(3,4)
b = torch.argmax(a, dim=0)
b

tensor([2, 2, 2, 2])

In [11]:
#Weigth coef for the Unsupervised
def dice_loss(logits, targets, is_modelT_preds=True): 
  if is_modelT_preds:
    targets_animal = torch.unsqueeze(targets, dim=1)
  targets_animal = torch.squeeze(targets)
  preds_animal = F.softmax(logits, dim=1)
  preds_animal = preds_animal[:,1,:,:]
  eps = 1e-6
  intersection = (preds_animal * targets_animal).sum()
  dice_coef = (2. * intersection + eps) / ((preds_animal**2).sum() + (targets_animal**2).sum() + eps)
  dice_loss = 1 - dice_coef
  return dice_loss

@torch.no_grad()
def wt(rampup_length, current, alpha, wait_period = 5):

  if current < wait_period:
    return 0.0
    
  else:
    if rampup_length == 0:
                return 1.0
    else:
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(alpha * np.exp(-5.0 * phase * phase))


#update the Teacher weight
@torch.no_grad()
def update_ema_variables(model, ema_model, alpha, global_step): 
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(alpha).add_(1 - alpha, param.data)


def unsup_loss(z, modelT, imgs, unsup_crit): 

  imgT_aug = augmentation(imgs).type(torch.float32)
  imgT_aug = imgT_aug.to(device)

  # z_bar will act as pseudo-labels
  with torch.no_grad():
    z_bar = modelT(imgT_aug)
    # z_bar = F.softmax(z_bar, dim = 1)
    z_bar_preds = torch.argmax(z_bar, dim=1)

  # Transform z_bar into predictions
  # Lu = unsup_crit(z, z_bar_preds).to(device)
  Lu = dice_loss(z, z_bar_preds, is_modelT_preds=True)

  return Lu

@torch.no_grad()
def evaluate_model(model, dataloader, device):
  
  model.eval()
  intersection_total, union_total = 0, 0
  pixel_correct, pixel_count = 0, 0
    
  for data in dataloader:
    imgs, labels = data
    imgs, labels = imgs.to(device), labels.to(device)
    logits = model(imgs)
    preds = torch.argmax(logits, dim=1)
    targets = torch.squeeze(labels)
            
    intersection_total += torch.logical_and(preds, targets).sum()
    union_total += torch.logical_or(preds, targets).sum()
            
    pixel_correct += (preds == targets).sum()
    pixel_count += targets.numel()

  iou = (intersection_total / union_total).item()
  accuracy = (pixel_correct / pixel_count).item()
  
  model.train()
  return accuracy, iou

In [12]:
#### Initialisation ####
#create 2 network
modelS = model_UNet.UNet(in_channels=3, num_classes=2, depth=depth)
modelS = modelS.to(device)
modelT = model_UNet.UNet(in_channels=3, num_classes=2, depth=depth)
modelT = modelT.to(device)
#create the losses
sup_crit = nn.CrossEntropyLoss().to(device)
unsup_crit = nn.CrossEntropyLoss().to(device)
#optimizer
optimizer = Adam(modelS.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=lr_gamma, last_epoch=-1, verbose=True)


Adjusting learning rate of group 0 to 1.0000e-03.


/content/drive/MyDrive/Colab Notebooks/COMP0197/comp0197-cw2/model_UNet.py:213: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
/content/drive/MyDrive/Colab Notebooks/COMP0197/comp0197-cw2/model_UNet.py:214: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


In [13]:
##data loader
mixed_train_loader, val_loader, test_loader = get_data(supervised_percent,1-supervised_percent,0.2,0.1, batch_size=batch_size, img_resize=img_resize, is_mixed_loader=is_mixed_labels)

all images are =  7393


In [ ]:
# continue from a previous trained model

# modelS = model_UNet.UNet(num_classes=2, in_channels=3, depth=3)
# modelS.load_state_dict(torch.load('models/unet_supervised_depth3_v1'))
# modelS.to(device)
# optimizer = Adam(modelS.parameters(), lr=lr)
# scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=lr_gamma, last_epoch=-1, verbose=True)

In [ ]:
# Train
eval_freq = 1
losses, accs, IOUs = [], [], []

for epoch in range(epochs):

        modelS.train()
        running_loss = 0
        running_loss_sup = 0
        running_loss_unsup = 0

        w_t = wt(rampup_length=ramp_up, current=epoch, alpha=alpha, wait_period=wait_period)

        for step, data in enumerate(mixed_train_loader):

            imgs, labs = data
            # Augment images
            imgS_aug = augmentation(imgs)

            imgS_aug = imgS_aug.to(device)
            labs = labs.squeeze().type(torch.LongTensor).to(device)

            optimizer.zero_grad()

            # Forward pass for student and teacher
            z = modelS(imgS_aug) 

            # Find img with label
            sup_idx = np.asarray(torch.tensor([(elem != -1).item() for elem in labs[:, 0, 0]])) #If batchsize is the first dim

            if sup_idx.sum() == 0:
              print('sup_idx == 0')
              Ls = torch.tensor(0)
            else:
              # Calculate losses
              # Ls = sup_crit(z[sup_idx], labs[sup_idx])
              Ls = dice_loss(z[sup_idx], labs[sup_idx])
            
            Lu = unsup_loss(z, modelT, imgs, unsup_crit)
    
            loss = Ls + w_t * Lu
            
            loss.backward()
            
            optimizer.step()    
            global_step += 1
            update_ema_variables(modelS, modelT, alpha, global_step)
            running_loss += loss.item()
            running_loss_sup += Ls.item()
            running_loss_unsup += Lu.item()
            

            #optimizer.param_groups[0]['lr'] = lr(epoch+2)
            #print(loss.item())

        print(f'Epoch: {epoch + 1:4d} - Loss: {running_loss:6.2f}, loss_sup: {running_loss_sup:6.1f}, loss_unsup: {running_loss_unsup:6.1f}, w_t: {w_t: 3.2f}')
        losses.append(running_loss)

        if (epoch % eval_freq == 0):
          accuracy, IOU = evaluate_model(modelS, val_loader, device)
          accs.append(accuracy)
          IOUs.append(IOU)
          print(f'accuracy: {accuracy:2.0%}; IOU: {IOU:2.0%}')

np.savetxt("losses", losses)
np.savetxt("accs", accs)
np.savetxt("IOUs", IOUs)

<ipython-input-11-36b9893a5883>:35: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  ema_param.data.mul_(alpha).add_(1 - alpha, param.data)


Epoch:    1 - Loss:  41.71, loss_sup:   41.7, loss_unsup:   18.6, w_t:  0.00
accuracy: 78%; IOU: 57%
Epoch:    2 - Loss:  34.56, loss_sup:   34.6, loss_unsup:   17.6, w_t:  0.00
accuracy: 72%; IOU: 58%
Epoch:    3 - Loss:  33.87, loss_sup:   33.9, loss_unsup:   17.3, w_t:  0.00
accuracy: 81%; IOU: 63%
Epoch:    4 - Loss:  31.74, loss_sup:   31.7, loss_unsup:   16.8, w_t:  0.00
accuracy: 82%; IOU: 64%
Epoch:    5 - Loss:  30.34, loss_sup:   30.3, loss_unsup:   16.9, w_t:  0.00
accuracy: 83%; IOU: 67%
Epoch:    6 - Loss:  34.47, loss_sup:   30.2, loss_unsup:   14.8, w_t:  0.29
accuracy: 82%; IOU: 67%
Epoch:    7 - Loss:  35.51, loss_sup:   29.4, loss_unsup:   13.7, w_t:  0.45
accuracy: 82%; IOU: 64%
Epoch:    8 - Loss:  37.39, loss_sup:   29.6, loss_unsup:   12.2, w_t:  0.64
accuracy: 82%; IOU: 67%
Epoch:    9 - Loss:  37.74, loss_sup:   28.3, loss_unsup:   11.6, w_t:  0.82
accuracy: 82%; IOU: 67%
Epoch:   10 - Loss:  39.15, loss_sup:   28.6, loss_unsup:   11.1, w_t:  0.95
accuracy: 80%;